Problem Statement. <br/>

Given strings S and T, find the minimum (contiguous) substring W of S, so that T is a subsequence of W. <br/>
If there is no such window in S that covers all characters in T, return the empty string "". If there are multiple such minimum-length windows, return the one with the left-most starting index. <br/>

Example 1: <br/>
Input: <br/>
S = "abcdebdde", T = "bde" <br/>
Output: "bcde" <br/>
Explanation:<br/>
"bcde" is the answer because it occurs before "bdde" which has the same length. <br/>
"deb" is not a smaller window because the elements of T in the window must occur in order.

# Top Down DP - O(N ^ 2) runtime, O(N ^ 2) space - Time Limit Exceeded

In [1]:
from typing import List

class Solution:
    def minWindow(self, S: str, T: str) -> str:
        n = len(S)
        dp = [[[float('-inf'), float('inf')] for _ in range(n)] for _ in range(n)]
        minLength = float('inf')
        res = ""
        
        for i in range(n):
            currLength = self.minWindowRecursive(dp, S, T, i, 0)
            if currLength < minLength:
                minLength = currLength
                res =  S[i: dp[i][0][1]]
            
        return res
    
    def minWindowRecursive(self, dp: List[List[int]], S: str, T: str, s_index: int, t_index: int) -> str:
        if (t_index == 0 and S[s_index] != T[t_index]) or s_index == len(S):
            return float('inf')
        
        if t_index == len(T) - 1 and S[s_index] == T[t_index]:
            if len(T) == 1:
                dp[s_index][t_index] = [s_index, s_index + 1]
            return 1
 
        if dp[s_index][t_index] == [float('-inf'), float('inf')]:
            
            if S[s_index] == T[t_index]:
                currentOrder = 1 + self.minWindowRecursive(dp, S, T, s_index + 1, t_index + 1)
            else:
                currentOrder = 1 + self.minWindowRecursive(dp, S, T, s_index + 1, t_index)

            dp[s_index][t_index] = [s_index, s_index + currentOrder]
            
            return currentOrder
        
        else:
            return dp[s_index][t_index][1] - dp[s_index][t_index][0]

# Top Down DP - O(N ^ 2) runtime, O(1) space - Time Limit Exceeded

In [4]:
class Solution:
    def minWindow(self, S: str, T: str) -> str:
        n = len(S)
        minLength = float('inf')
        start = -1
        res = ""
        
        for i in range(n):
            currLength = self.minWindowRecursive(S, T, i, 0)
            if currLength < minLength:
                minLength = currLength
                start = i
            
        return S[start: start + minLength] if start > -1 else ""
    
    def minWindowRecursive(self, S: str, T: str, s_index: int, t_index: int) -> str:
        if (t_index == 0 and S[s_index] != T[t_index]) or s_index == len(S):
            return float('inf')
        
        if t_index == len(T) - 1 and S[s_index] == T[t_index]:
            return 1
            
        if S[s_index] == T[t_index]:
            currentOrder = 1 + self.minWindowRecursive(S, T, s_index + 1, t_index + 1)
        else:
            currentOrder = 1 + self.minWindowRecursive(S, T, s_index + 1, t_index)

        return currentOrder

# Dict and Set - O(S ^ 2 * T) runtime, O(S + T) space - Time Limit Exceeded

In [6]:
from collections import defaultdict

class Solution:
    def minWindow(self, S: str, T: str) -> str:
        n = len(S)
        minLength = float('inf')
        start = -1
        res = ""
        t_set = set(list(T))
            
        s_dict = defaultdict(list)
        for i, char in enumerate(S):
            if char in t_set:
                s_dict[char].append(i)
                
        if len(t_set) > len(s_dict): return ""
        
        for i in range(n):
            if S[i] == T[0]:
                found = True
                currIndex = i
                for j, char in enumerate(T[1:]):
                    found = False
                    for k in s_dict[char]:
                        if k > currIndex:
                            found = True
                            currIndex = k
                            break
                    if not found:
                        break
                if found and currIndex - i + 1 < minLength:
                    minLength = currIndex - i + 1
                    start = i
            
        return S[start: start + minLength] if start > -1 else ""

# DP (Postfix Variation)- O(S * T) runtime, O(S) space

In [8]:
class Solution:
    def minWindow(self, S: str, T: str) -> str:
        cur = [i if x == T[0] else None
               for i, x in enumerate(S)]
        #At time j when considering T[:j+1],
        #the smallest window [s, e] where S[e] == T[j]
        #is represented by cur[e] = s.
        for j in range(1, len(T)):
            last = None
            new = [None] * len(S)
            #Now we would like to calculate the candidate windows
            #"new" for T[:j+1].  'last' is the last window seen.
            for i, u in enumerate(S):
                if last is not None and u == T[j]: new[i] = last
                if cur[i] is not None: last = cur[i]
            cur = new

        #Looking at the window data cur, choose the smallest length
        #window [s, e].
        ans = 0, len(S)
        for e, s in enumerate(cur):
            if s is not None and s >= 0 and e - s < ans[1] - ans[0]:
                ans = s, e
        return S[ans[0]: ans[1]+1] if ans[1] < len(S) else ""

# DP (Next Array Variation) - O(S * T) runtime, O(S) space

In [10]:
class Solution:
    def minWindow(self, S: str, T: str) -> str:
        N = len(S)
        nxt = [None] * N
        last = [-1] * 26
        for i in range(N-1, -1, -1):
            last[ord(S[i]) - ord('a')] = i
            nxt[i] = tuple(last)

        windows = [[i, i] for i, c in enumerate(S) if c == T[0]]
        for j in range(1, len(T)):
            letter_index = ord(T[j]) - ord('a')
            windows = [[root, nxt[i+1][letter_index]]
                       for root, i in windows
                       if 0 <= i < N-1 and nxt[i+1][letter_index] >= 0]

        if not windows: return ""
        i, j = min(windows, key=lambda x: x[1]-x[0])
        return S[i: j+1]

In [11]:
instance = Solution()
instance.minWindow("abcdebdde", "bde")

'bcde'